In [ ]:
pip install transformers

In [ ]:
pip install huggingface-hub

In [ ]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 11.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-chat-hf', load_in_8bit=True)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokr = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
prompt = "Hsoub Limited company is a "
toks = tokr(prompt, return_tensors="pt")

In [ ]:
toks

{'input_ids': tensor([[    1,   379, 29879,   283, 29890, 28873,  5001,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
tokr.batch_decode(toks['input_ids'])

['<s> Hsoub Limited company is a ']

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15)
res

CPU times: user 5.56 s, sys: 564 ms, total: 6.13 s
Wall time: 10.2 s


tensor([[    1,   379, 29879,   283, 29890, 28873,  5001,   338,   263, 29871,
         29896, 29900, 29900, 29995, 11684,  8819,   653,   310,   379, 29879,
           283, 29890,  6431, 29892,   263]], device='cuda:0')

In [ ]:
tokr.batch_decode(res)

['<s> Hsoub Limited company is a 100% subsidiary of Hsoub Group, a']

In [ ]:
def gen(p, maxlen=15):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen)
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 30)

['<s> Hsoub Limited company is a 100% export-oriented enterprise specializing in the design, development, manufacture and sales of various types of industrial valves,']

In [ ]:
gen("The sum of the numbers 1 and 25 equal to", 5)

['<s> The sum of the numbers 1 and 25 equal to 26, how']

In [ ]:
gen("write a python code to sum two numbers", 40)

['<s> write a python code to sum two numbers\n\nWrite a Python function that takes two arguments, `a` and `b`, and returns their sum.\n\nFor example, if you call the function with `a = 3`']

In [ ]:
gen("Write a function in Python that adds two numbers and returns their sum", 60)

['<s> Write a function in Python that adds two numbers and returns their sum\n\nWrite a function in Python that adds two numbers and returns their sum. The function should take two arguments, num1 and num2, and return their sum.\n\nHere is an example of how you might implement this function:\n```\ndef add_numbers(num1, num']

In [ ]:
gen("Write a function in Python that adds two numbers and returns their sum", 200)

["<s> Write a function in Python that adds two numbers and returns their sum.\n\nFor example, if you call the function with the arguments `a = 3`, `b = 4`, the function should return `7`.\n\nHow would you approach this problem?\n\n```\ndef add(a, b):\n    # code here\n    return sum\n```\n\nYou can use the built-in `sum` function to add the two numbers.\n\nHere's an example of how you could implement the `add` function:\n```\ndef add(a, b):\n    return sum(a, b)\n```\nThis will return the sum of `a` and `b`.\n\nYou can also use the `+` operator to add the two numbers directly:\n```\ndef add(a, b):\n    return a + b\n```\nThis will also return the sum of `a` and `b`.\n\nYou can test the function with some examples:\n```"]

In [ ]:
response = gen("Write a function in Python that adds two numbers and returns their sum", 200)

In [ ]:
decoded_string = response[0].encode('utf-8').decode('unicode-escape')
print(decoded_string)

<s> Write a function in Python that adds two numbers and returns their sum

Write a function in Python that adds two numbers and returns their sum. Here is an example of how you might do this:

def add_numbers(a, b):
    return a + b

Now, try running the function by calling it with two numbers as arguments, like this:

print(add_numbers(3, 5)) # Output: 8

This will print the sum of 3 and 5, which is 8.

You can also test the function with different inputs to make sure it works correctly. For example:

print(add_numbers(1, 2)) # Output: 3
print(add_numbers(4, 6)) # Output  Output: 10

As you can see, the function works correctly for any two numbers you give it.

Here are some tips for writing functions in Python:

* Functions should have a clear


In [ ]:
gen("ماهي لغة بايثون", 40)

['<s> ماهي لغة بايثون بيرسون للتعليم العام 2017\n\nماهي لغة بايثو']

In [ ]:
response = gen("اكتب دالة بلغة بايثون تجمع رقمين وتعيد مجموعهما", 600)

In [ ]:
text_with_newlines = response[0].replace("\\n", "\n")

print(text_with_newlines)

<s> اكتب دالة بلغة بايثون تجمع رقمين وتعيد مجموعهما باستخدام مكانة واحدة للرقمين وللمجموعهما باستخدام مكانة واحدة للرقمين وللمجموعهما

وينبغي أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أيضا أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أيضا أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أيضا أن تكون مكانة واحدة للرقمين وللمجموعهما من مكانات مختلفة في المجموعة.

وينبغي أن تكون مكانة واحدة للرقمين وللمجموعهم
